# Machine learning using EKF

In [62]:
from IPython import display
import os
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from load import load_abalone_data,load_bikes_data
# Importing Pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from  sklearn.datasets import make_regression
from sklearn.datasets import load_boston
from tqdm import tqdm as tqdm

In [63]:
# Select device which you are going to use for training
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

cpu


### Import Data Sets
Testing using a toy sine data

In [64]:
X,y = load_bikes_data()
print(y.shape)
print(X.shape)

(17379,)
(17379, 14)


## Data Partition


In [65]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,test_size=.5)
X_val,X_test, y_val,y_test = train_test_split(X_test,y_test,test_size=0.5)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
# x_train_scaled = scaler.fit_transform(x_train)
# x_test_scaled = scaler.transform(x_test)

(8689, 14)
(4345, 14)
(4345, 14)


## Define Neural network

In [67]:
class MLP(nn.Module):
    def __init__(self, n_inputs, n_hidden_layer, n_outputs,bias=True):
        super(MLP, self).__init__()
        # YOUR CODE HERE
        #raise NotImplementedError()
        self.fc1 = nn.Linear(n_inputs, n_hidden_layer, bias)
        self.fc2 = nn.Linear(n_hidden_layer, n_hidden_layer, bias)
        self.fc3 = nn.Linear(n_hidden_layer, n_hidden_layer, bias)
        self.fc4 = nn.Linear(n_hidden_layer, n_outputs, bias)
        

    def forward(self, x):
        # YOUR CODE HERE
        #raise NotImplementedError()
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x    

In [68]:
## Testing net
n_inputs = X.shape[1]
n_outputs = 1
n_hidden_layer = 100
test_net = MLP(n_inputs, n_hidden_layer, n_outputs)
print(test_net)

MLP(
  (fc1): Linear(in_features=14, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=1, bias=True)
)


In [69]:
# Define number of Input and Output layers
torch.set_default_dtype(torch.float64)
n_inputs = X.shape[1]
n_outputs = 1
n_hidden_layer = 20
mlp = MLP(n_inputs,n_hidden_layer, n_outputs)
mpl = mlp.to(device)

optimizer = torch.optim.Adam(mlp.parameters(), lr=0.005)
n_epochs = 200

train_accuracy_history = []
test_accuracy_history = []
print_every = 500
x_tensor = torch.tensor(X_train, device=device, dtype=torch.float64)
y_tensor = torch.tensor(y_train, device=device, dtype=torch.float64)
x_val_tensor = torch.tensor(X_val, device=device, dtype=torch.float64)
print_every = 10
ceLoss = nn.MSELoss()



for epoch in (range(n_epochs)):
    
    #Set gradients as zero
    mlp.zero_grad()
    #Calculate outputs with forward()
    outputs = mlp(x_tensor)   
    
    #calculate loss
    loss = ceLoss(outputs, y_tensor)
#     print(f"The output is :{outputs.item()} expected is :{y_train}")
    #calculate gradients - calling backward()
    loss.backward()
    
    #update weights using optimizer
    optimizer.step()
    if(epoch%print_every ==0):
        with torch.no_grad():
            y_pred = mlp.forward(x_val_tensor)
            y_pred = y_pred.cpu().data.numpy()
            error = mean_squared_error(y_val,y_pred)
            val_error =  np.sqrt(error)
        print(f"sample:{epoch} train error: {loss.item()} val error:{val_error.item()}")

sample:0 train error: 68962.73798949139 val error:263.1950145905543
sample:10 train error: 68826.19828299053 val error:262.9173268352678
sample:20 train error: 68464.09566512985 val error:262.09234444164997
sample:30 train error: 67193.74032118951 val error:259.1660500509707
sample:40 train error: 63440.46636641721 val error:250.40093538410608
sample:50 train error: 54741.997558990464 val error:229.01455214006597
sample:60 train error: 40927.833787824944 val error:190.685903182969
sample:70 train error: 34092.03856734416 val error:167.36815957964083
sample:80 train error: 34630.3041933079 val error:167.73295479563498
sample:90 train error: 33961.386520177446 val error:170.3560687421684
sample:100 train error: 33893.99672271912 val error:169.93964124630114
sample:110 train error: 33788.410499905935 val error:168.98565423107354
sample:120 train error: 33733.87463025582 val error:169.54837438986112
sample:130 train error: 33678.38472672038 val error:170.16270162299406
sample:140 train err

In [70]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
with torch.no_grad():
    x_test = torch.tensor(X_test, device=device, dtype=torch.float64)
    y_pred = mlp.forward(x_test)
    y_pred = y_pred.cpu().data.numpy()
    error = mean_squared_error(y_test,y_pred)
    print(np.sqrt(error))

173.80891063683873
